### Test of data instantiation

In [40]:
import torch
import numpy
import matplotlib as plt
from torchvision.utils import draw_bounding_boxes
from torch.utils.data import Dataset, DataLoader 
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from torchvision.transforms import functional as F
import pandas as pd
from PIL import Image
import os
from src import BWDatasets

In [4]:
from paddleocr import PaddleOCR as POCR

ocr = POCR(lang="en", use_doc_orientation_classify=True, use_doc_unwarping=True, use_textline_orientation=True)


c:\Users\legom\anaconda3\envs\birdword\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\legom\anaconda3\envs\birdword\Lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in C:\Users\legom\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 3727.72it/s]
Creating model: ('UVDoc', None)
Using official model (UVDoc), the model files will be automatically downloaded and saved in C:\Users\le

In [2]:
label_path = "dataset/datasets/rf/ringcodes.csv"
image_path = "dataset/datasets/rf/images"
max_n = 10
transform = T.Compose([
    #T.Resize((224,224)),
    T.ToTensor()
])
exp_dataset = BWDatasets.TrainDataSet(img_path=image_path, labels_path=label_path, transform=transform, max_n=max_n)


In [ ]:

# img_crop = F.crop()


In [36]:
# 0 0.521075 0.558316 0.0353341 0.104376 0.83813

def calculate_bb_cords(image, bb):
    bb_x = image.shape[2] * float(bb[1])
    bb_y = image.shape[1] * float(bb[2])
    bb_w = image.shape[2] * float(bb[3])
    bb_h = image.shape[1] * float(bb[4])

    return bb_x, bb_y, bb_w, bb_h


In [25]:
def bb_txt_to_list(bb_path):
    
    with open(bb_path) as f:
        line = f.readline().strip()

        bb = line.split(' ')
    
    return bb

In [30]:
img, l = exp_dataset[0]
bb_path = "dataset/datasets/rf/labels/20240408-145102.064779(162.29,-4.57,9209.0).txt"
bb = bb_txt_to_list(bb_path)
print(bb)

['0', '0.521075', '0.558316', '0.0353341', '0.104376', '0.83813']


In [39]:
img, l = exp_dataset[0]

bb_cords = calculate_bb_cords(img, bb)

print(bb_cords)


(1000.4639999999999, 602.9812800000001, 67.841472, 112.72608)


In [ ]:

img_uint8 = (img * 255).to(torch.uint8)

# 2) prepare your box(es): tensor of shape (num_boxes, 4) in (xmin, ymin, xmax, ymax)
boxes = torch.tensor([[bb_cords[0], bb_cords[1], bb_cords[0]+bb_cords[2], bb_cords[1]+bb_cords[3]]], dtype=torch.int)

# 4) draw
img_boxes = draw_bounding_boxes(
    img_uint8,
    boxes,
    labels=labels,
    colors="red",
    width=2
)

# 5) convert back to PIL or plot with matplotlib
plt.imshow(img_boxes)
plt.axis("off")
plt.show()




In [3]:

exp_loader = DataLoader(dataset=exp_dataset, batch_size=64, shuffle=True)


In [23]:
def prediction(dataloader):
    for images, labels in dataloader:
        images = BWDatasets.tensor_to_numpy(images)
        labels = labels[0]
        for i, image in enumerate(images):
            result = ocr.predict(image)

            print("-------------------")
            print(f'Prediction: {result[0]["rec_texts"]}')
            print(f'Actual: {labels[i]}')



In [24]:
prediction(exp_loader)



-------------------
Prediction: []
Actual: JJ94C
-------------------
Prediction: ['']
Actual: JE24X
-------------------
Prediction: []
Actual: JE25X
-------------------
Prediction: []
Actual: JJ89C
-------------------
Prediction: []
Actual: J808V
-------------------
Prediction: []
Actual: J194V
-------------------
Prediction: []
Actual: JE24X
-------------------
Prediction: []
Actual: JJ94C
-------------------
Prediction: []
Actual: JE94C
-------------------
Prediction: []
Actual: JE24X
